<a href="https://colab.research.google.com/github/POOJACH76/Hair_Loss_Prediction/blob/main/Hair_Loss__Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz


In [2]:
df = pd.read_csv("/content/Luke_hair_loss_documentation.csv")

In [3]:
df.head(5)


,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,shampoo_brand,swimming,hair_washing,dandruff
0,Few,2,Low,0,1,Pantene,No,Y,None
1,Few,0,Low,0,3,Pantene,No,N,None
2,Medium,3,Low,1,0,Pantene,Yes,Y,None
3,Few,2,Low,0,1,Pantene,No,N,None
4,Few,2,Low,0,1,Pantene,No,Y,None


In [4]:
df.columns

Index(['hair_loss', 'stay_up_late', 'pressure_level', 'coffee_consumed',
       'brain_working_duration', 'shampoo_brand', 'swimming', 'hair_washing',
       'dandruff'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   hair_loss               400 non-null    object
 1   stay_up_late            400 non-null    int64 
 2   pressure_level          400 non-null    object
 3   coffee_consumed         400 non-null    int64 
 4   brain_working_duration  400 non-null    int64 
 5   shampoo_brand           400 non-null    object
 6   swimming                400 non-null    object
 7   hair_washing            400 non-null    object
 8   dandruff                400 non-null    object
dtypes: int64(3), object(6)
memory usage: 28.2+ KB


In [6]:

print(df['hair_loss'].unique())
print(df['pressure_level'].unique())
print(df['shampoo_brand'].unique())
print(df['dandruff'].unique())


['Few' 'Medium' 'Many' 'A lot']
['Low' 'Medium' 'High' 'Very High']
['Pantene' 'Hair & Shoulder']
['None' 'Few' 'Many']


In [7]:
df.head(5)

,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,shampoo_brand,swimming,hair_washing,dandruff
0,Few,2,Low,0,1,Pantene,No,Y,None
1,Few,0,Low,0,3,Pantene,No,N,None
2,Medium,3,Low,1,0,Pantene,Yes,Y,None
3,Few,2,Low,0,1,Pantene,No,N,None
4,Few,2,Low,0,1,Pantene,No,Y,None


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

categorical_columns = ['hair_loss','pressure_level','shampoo_brand','dandruff','hair_washing','swimming']

encoders = {}

for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    encoders[column] = le


with open('label_encoders.pkl', 'wb') as encoder_file:
    pickle.dump(encoders, encoder_file)


In [9]:
df['hair_loss'].unique()

array([1, 3, 2, 0])

In [10]:
X = df.drop("hair_loss", axis=1 )
y= df["hair_loss"]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   hair_loss               400 non-null    int64
 1   stay_up_late            400 non-null    int64
 2   pressure_level          400 non-null    int64
 3   coffee_consumed         400 non-null    int64
 4   brain_working_duration  400 non-null    int64
 5   shampoo_brand           400 non-null    int64
 6   swimming                400 non-null    int64
 7   hair_washing            400 non-null    int64
 8   dandruff                400 non-null    int64
dtypes: int64(9)
memory usage: 28.2 KB


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.20 ,random_state=45)
from sklearn import metrics

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=45)
rf_model.fit(X_train, y_train)

rf_y_pred = rf_model.predict(X_test)
rf_accuracy = metrics.accuracy_score(y_test, rf_y_pred)
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")

Random Forest Accuracy: 85.00%


In [14]:
import pickle
filename= "rf_model.pkl"

with open(filename, 'wb') as file:
    pickle.dump(rf_model, file)

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt

def get_scores(model, actual, predicted):
    print(f"Model: {model}")


    accuracy = metrics.accuracy_score(actual, predicted)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    conf_matrix = metrics.confusion_matrix(actual, predicted)
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

In [16]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_classifier = DecisionTreeClassifier(max_depth=5)
decision_tree_classifier.fit(X_train, y_train)
decision_tree_y_pred = decision_tree_classifier.predict(X_test)
decision_tree_accuracy = metrics.accuracy_score(y_test, decision_tree_y_pred)
print(f"Decision Tree Accuracy: {decision_tree_accuracy * 100:.2f}%")

Decision Tree Accuracy: 82.50%


In [17]:
from xgboost import XGBClassifier
xgboost_classifier = XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.1)
xgboost_classifier.fit(X_train, y_train)
xgboost_y_pred = xgboost_classifier.predict(X_test)

xgboost_accuracy = metrics.accuracy_score(y_test, xgboost_y_pred)
print(f"XGBoost Accuracy: {xgboost_accuracy * 100:.2f}%")


XGBoost Accuracy: 85.00%


In [18]:
from sklearn import metrics

rf_model_confusion_matrix = metrics.confusion_matrix(y_test, rf_y_pred)
decision_tree_confusion_matrix = metrics.confusion_matrix(y_test, decision_tree_y_pred)
xgb_model_confusion_matrix = metrics.confusion_matrix(y_test, xgboost_y_pred)
print(rf_model_confusion_matrix)
print(decision_tree_confusion_matrix)
print(xgb_model_confusion_matrix)

[[ 2  0  2  0]
 [ 0 36  0  2]
 [ 3  0  7  1]
 [ 0  4  0 23]]
[[ 2  0  2  0]
 [ 0 35  0  3]
 [ 3  0  7  1]
 [ 0  5  0 22]]
[[ 2  0  2  0]
 [ 0 37  0  1]
 [ 3  0  7  1]
 [ 0  5  0 22]]


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf_y_pred))
print(classification_report(y_test, decision_tree_y_pred))
print(classification_report(y_test, xgboost_y_pred))

              precision    recall  f1-score   support

           0       0.40      0.50      0.44         4
           1       0.90      0.95      0.92        38
           2       0.78      0.64      0.70        11
           3       0.88      0.85      0.87        27

    accuracy                           0.85        80
   macro avg       0.74      0.73      0.73        80
weighted avg       0.85      0.85      0.85        80

              precision    recall  f1-score   support

           0       0.40      0.50      0.44         4
           1       0.88      0.92      0.90        38
           2       0.78      0.64      0.70        11
           3       0.85      0.81      0.83        27

    accuracy                           0.82        80
   macro avg       0.72      0.72      0.72        80
weighted avg       0.83      0.82      0.82        80

              precision    recall  f1-score   support

           0       0.40      0.50      0.44         4
           1       0.

In [20]:
unseen_df = pd.read_csv('/content/Luke_hair_loss_documentation.csv')

In [21]:
unseen_df.head(10)

,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,shampoo_brand,swimming,hair_washing,dandruff
0,Few,2,Low,0,1,Pantene,No,Y,None
1,Few,0,Low,0,3,Pantene,No,N,None
2,Medium,3,Low,1,0,Pantene,Yes,Y,None
3,Few,2,Low,0,1,Pantene,No,N,None
4,Few,2,Low,0,1,Pantene,No,Y,None
5,Few,0,Low,0,3,Pantene,Yes,Y,None
6,Medium,3,Low,1,1,Pantene,Yes,Y,None
7,Few,0,Low,0,1,Pantene,No,N,None
8,Few,2,Low,0,2,Pantene,No,N,None
9,Medium,1,Low,1,2,Pantene,Yes,Y,None


In [24]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

with open('label_encoders.pkl', 'rb') as encoder_file:
    encoders = pickle.load(encoder_file)

unseen_df = pd.read_csv('/content/Hair Loss.csv')

categorical_columns = ['hair_loss', 'pressure_level', 'shampoo_brand', 'dandruff', 'hair_washing', 'swimming']
for column in categorical_columns:
    le = encoders[column]
    unseen_df[column] = le.transform(unseen_df[column])

with open('rf_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

x_unseen = unseen_df.drop(['hair_loss'], axis=1)
predictions = loaded_model.predict(x_unseen)
true_labels = unseen_df['hair_loss']
accuracy = accuracy_score(true_labels, predictions)
print("Accuracy:", accuracy)

confusion_matrix_unseen = confusion_matrix(true_labels, predictions)
print("Confusion Matrix for Unseen Data:")
print(confusion_matrix_unseen)

y_probs = loaded_model.predict_proba(x_unseen)
probability_df = pd.DataFrame(y_probs, columns=loaded_model.classes_)
probability_df['hair_loss'] = true_labels.values
print(probability_df)

prob_no = y_probs[:, 0]
prob_yes = y_probs[:, 1]
print(f"Probability of 'no': {prob_no[0] * 100:.2f}%")
print(f"Probability of 'yes': {prob_yes[0] * 100:.2f}%")
print(classification_report(true_labels, predictions))

Accuracy: 0.5714285714285714
Confusion Matrix for Unseen Data:
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 4]]
          0     1         2         3  hair_loss
0  0.280000  0.13  0.045000  0.545000          3
1  0.000000  0.23  0.260000  0.510000          0
2  0.356500  0.14  0.102659  0.400841          3
3  0.281333  0.19  0.061667  0.467000          3
4  0.198833  0.21  0.181667  0.409500          2
5  0.291000  0.17  0.095952  0.443048          1
6  0.292667  0.04  0.115000  0.552333          3
Probability of 'no': 28.00%
Probability of 'yes': 13.00%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.57      1.00      0.73         4

    accuracy                           0.57         7
   macro avg       0.14      0.25      0.18         7
weighted avg       0.33      0.57      0.42         7



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
